In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd ./gdrive/MyDrive/DOD\ Dataset

/content/gdrive/MyDrive/DOD Dataset


In [ ]:
!pip install transformers
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.0 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=89deac6e7c53190f0bedde139795d4010bc26de781deac5b8762ce4a4961847a
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
from torch.utils.data import Dataset, random_split
from pathlib import Path
from pandas import read_csv, concat
import numpy as np



class TypeDataset(Dataset):
    def __init__(self, data_dir: str):
        data_dir = Path(data_dir)

        type_df = read_csv(data_dir / "type_dataset.csv")
        self.len = len(type_df)
        self.examples = type_df

    def __getitem__(self, i):

        label = self.examples.iloc[i].label
        if label == "Policy":
            label = 0
        elif label == "Responsibility":
            label = 1
        elif label == "Procedures":
            label = 2

        return {
            "text": self.examples.iloc[i].text,
            "label": label,
        }

    def __len__(self):
        return len(self.examples)

In [ ]:
dataset = TypeDataset('./')

In [ ]:
len(dataset)

7698

In [ ]:
dataset[5000]

{'text': '  Each Federal Government department and agency shall ensure the assured sharing of information by implementing the following measures:  TechnologyIdentify and use, to the maximum extent applicable, Federal or industry best-practice AIS standards, technologies, and business processes that maximize the effectiveness of AIS. Use these applicable best practices to make NSS information available, as authorized, at all classification levels and across all infrastructures, whether virtual and/or under the direct control of the agency of origination that is providing and transferring the data.',
 'label': 0}

In [ ]:
policies = [x for x in dataset if x['label'] == 0]
responsibility = [x for x in dataset if x['label'] == 1]
procedures = [x for x in dataset if x['label'] == 2]

In [ ]:
print(len(policies), len(responsibility), len(procedures))

1531 4175 1992


In [ ]:
m_datasets = {}
m_datasets[0] = policies
m_datasets[1] = responsibility
m_datasets[2] = procedures

In [ ]:
import random

train_dataset = []
val_dataset = []
test_dataset = [] 


def getRandomFromData(data, percent):
    size = int((percent * len(data)) / 100)

    output = []
    for i in range(size):
        element = random.choice(data)
        output.append(element)
        data.remove(element)

    return output, data

num_types = len(m_datasets.keys())

for i in range(num_types):
    dat = m_datasets[i]
    train_d, dat = getRandomFromData(dat, 60)
    val_d, dat = getRandomFromData(dat, 15)    
    test_d, dat = getRandomFromData(dat, 25)

    train_dataset += train_d
    val_dataset += val_d
    test_dataset += test_d


# for i in range(len(responsibility)):
#     if i < len(policies) - 600:
#         train_dataset.append(policies[i])
    
#     elif i < len(policies) - 300:
#         val_dataset.append(policies[i])
    
#     elif i < len(policies):
#         test_dataset.append(policies[i])

#     if i < len(procedures) - 600:
#         train_dataset.append(procedures[i])
    
#     elif i < len(procedures) - 300:
#         val_dataset.append(procedures[i])
    
#     elif i < len(procedures):
#         test_dataset.append(procedures[i])
    

#     if i < len(responsibility) - 1000:
#         train_dataset.append(responsibility[i])
    
#     elif i < len(responsibility) - 600:
#         val_dataset.append(responsibility[i])
    
#     elif i < len(responsibility):
#         test_dataset.append(responsibility[i])


In [ ]:
print(len(train_dataset), len(val_dataset), len(test_dataset))

4618 460 654


In [ ]:
policies = [x for x in test_dataset if x['label'] == 0]
responsibility = [x for x in test_dataset if x['label'] == 1]
procedures = [x for x in test_dataset if x['label'] == 2]

In [ ]:
print(len(policies), len(responsibility), len(procedures))

130 355 169


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from datasets import load_dataset, Dataset, DatasetDict

# mydataset = Dataset.from_generator(dataset) 

myTraindataset = Dataset.from_list(train_dataset)
myValdataset = Dataset.from_list(val_dataset)
myTestdataset = Dataset.from_list(test_dataset)


mydataset = DatasetDict({"train": myTraindataset, "validation": myValdataset, "test":myTestdataset})

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

#raw_datasets = load_dataset("glue", "mrpc")

# facebook/bart-base
#"bert-base-uncased"
# roberta-base
# nlpaueb/legal-bert-base-uncased
# casehold/custom-legalbert


checkpoint = "mukund/privbert"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)


tokenized_datasets = mydataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--mukund--privbert/snapshots/48228b4661fa8252bdb39ca44a4d9758f6b37f88/config.json
Model config RobertaConfig {
  "_name_or_path": "mukund/privbert",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.28.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--mukund--privbert/snapshots/48228b4661fa8252bd

Map:   0%|          | 0/4618 [00:00<?, ? examples/s]

Map:   0%|          | 0/460 [00:00<?, ? examples/s]

Map:   0%|          | 0/654 [00:00<?, ? examples/s]

In [ ]:
mydataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 4618
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 460
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 654
    })
})

In [ ]:
mydataset['train'][0]

{'text': 'The DoD Components shall seek and follow spectrum supportability guidance from the Military Communications-Electronics Board (MCEB) prior to assuming any contractual obligations for the full-scale development, production, procurement, or deployment of spectrum dependent (i.e., wireless) devices or systems, in accordance with DoD Directive 4650.01 (reference (h)).',
 'label': 0}

In [ ]:
from transformers import TrainingArguments, Trainer, logging

# training_args = TrainingArguments("test-trainer")

default_args = {
    "output_dir": "./",
    "evaluation_strategy": "epoch",
    "num_train_epochs": 6,
    # "log_level": "error",
    "logging_steps": 1,
    "log_level" : "info",
    "report_to": "none",
}

training_args = TrainingArguments(per_device_train_batch_size=64,
                                  # gradient_accumulation_steps=32,
                                  gradient_checkpointing=True,
                                  # optim="adafactor",
                                  **default_args,
                                )

PyTorch: setting up devices


In [ ]:
training_args

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=True,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=every_save,
hub_token=<HUB_TOKEN>,
igno

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--mukund--privbert/snapshots/48228b4661fa8252bdb39ca44a4d9758f6b37f88/config.json
Model config RobertaConfig {
  "_name_or_path": "mukund/privbert",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.28.1",
  "type_vocab_size": 1,
  "use_cache

In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from datasets import load_metric
from sklearn.metrics import mean_squared_error
import evaluate
import numpy as np


accuracy = evaluate.load("accuracy")

# imdb = load_dataset("imdb")

f1 = evaluate.load("f1")

# imdb = load_dataset("imdb")


#F1
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    microF1 = f1.compute(predictions=predictions, references=labels, average='micro')
    macroF1 = f1.compute(predictions=predictions, references=labels, average='macro')
    weightF1 = f1.compute(predictions=predictions, references=labels, average='weighted')

    return {"micro-F1": microF1, "macroF1": macroF1, "weightF1": weightF1}


# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     predictions = np.argmax(predictions, axis=1)
#     return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import Trainer, TrainerCallback
import torch


trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
    # callbacks=[GPUCallback()]
)

In [ ]:
!nvidia-smi

Mon May  1 21:32:32 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P0    33W /  70W |  13413MiB / 15360MiB |     11%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4,618
  Num Epochs = 6
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 438
  Number of trainable parameters = 124,647,939
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than u

Epoch,Training Loss,Validation Loss,Micro-f1,Macrof1,Weightf1
1,0.564100,0.257127,{'f1': 0.9108695652173913},{'f1': 0.8833182273182273},{'f1': 0.912013792448575}
2,0.011700,0.200058,{'f1': 0.9326086956521739},{'f1': 0.9107040939900415},{'f1': 0.9338942763114436}
3,0.102600,0.215418,{'f1': 0.9434782608695652},{'f1': 0.9268442217191409},{'f1': 0.9439697739970037}
4,0.000900,0.236796,{'f1': 0.9478260869565217},{'f1': 0.9307912970563573},{'f1': 0.9479273616203946}
5,0.000600,0.228322,{'f1': 0.9565217391304348},{'f1': 0.9446255108839399},{'f1': 0.9570060986494514}
6,0.000700,0.257966,{'f1': 0.9565217391304348},{'f1': 0.9437868381889863},{'f1': 0.9569832595913328}


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 460
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 460
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 460
  Batch size = 8
T

TrainOutput(global_step=438, training_loss=0.12417860002071941, metrics={'train_runtime': 2039.8741, 'train_samples_per_second': 13.583, 'train_steps_per_second': 0.215, 'total_flos': 4533395881474404.0, 'train_loss': 0.12417860002071941, 'epoch': 6.0})

In [ ]:
trainer.evaluate(eval_dataset=tokenized_datasets["test"])

The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 654
  Batch size = 8


{'eval_loss': 0.18646521866321564,
 'eval_micro-F1': {'f1': 0.9694189602446482},
 'eval_macroF1': {'f1': 0.9579999931463266},
 'eval_weightF1': {'f1': 0.9693848031416217},
 'eval_runtime': 6.4119,
 'eval_samples_per_second': 101.997,
 'eval_steps_per_second': 12.789,
 'epoch': 6.0}

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, r2_score, explained_variance_score
import numpy as np

In [ ]:
eval_dataset = tokenized_datasets["test"]
predictions = trainer.predict(eval_dataset)

In [ ]:
predictions.metrics

In [ ]:
y_true = [np.argmax(p) for p in predictions.predictions]
y_pred = predictions.label_ids

In [ ]:
target_names = ['policies', 'responsibility', 'procedures']

print(classification_report(y_true, y_pred, target_names=target_names))

In [ ]:
trainer.save_model()

Saving model checkpoint to ./
Configuration saved in ./config.json
Model weights saved in ./pytorch_model.bin
tokenizer config file saved in ./tokenizer_config.json
Special tokens file saved in ./special_tokens_map.json


In [ ]:
from transformers import Trainer, TrainerCallback
import torch


trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
    # callbacks=[GPUCallback()]
)

In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1200
  Batch size = 8


{'eval_loss': 0.9670817255973816,
 'eval_micro-F1': {'f1': 0.75},
 'eval_macroF1': {'f1': 0.654678610159748},
 'eval_weightF1': {'f1': 0.7139768919971413},
 'eval_runtime': 9.263,
 'eval_samples_per_second': 129.547,
 'eval_steps_per_second': 16.193}

In [ ]:
!pip install captum
!pip install transformers-interpret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 KB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.6 MB/s eta 0:00:00


In [ ]:
train_dataset[1551]

{'text': "\n777 Relax Spa\n4310 w lllinois Ave #240. Midland. TX 79703\nOpen 7 days 9:00am-10:00pm\nTel: 432-934-2797\nNew and beautiful ladies waiting to pamper you with the best skills available anywhere.\nWe are highly trained in the art of pleasure and we know how to please even the most demanding man.\nWe will rub our delicious bodies all over you as we massage you with the pleasure that will blow your mind.\nWe know what you're looking for and we are here to give it to you.\nWe are the beauties you've been searching for.\nWe're not clock watchers like the other places.\nYou won't be disappointed when you see us,that's a promise.\nYou'll be even happier with the wonderful service we offer.\nstop by today and give us a try .\n\xa0\n\xa0\n\n\n\n\nWe are an agency and serve many cities. Photos are a likeness of the girls presented but may not be 100% accurate.",
 'label': 0}

In [ ]:
text= train_dataset[1551]['text']
text

"\n777 Relax Spa\n4310 w lllinois Ave #240. Midland. TX 79703\nOpen 7 days 9:00am-10:00pm\nTel: 432-934-2797\nNew and beautiful ladies waiting to pamper you with the best skills available anywhere.\nWe are highly trained in the art of pleasure and we know how to please even the most demanding man.\nWe will rub our delicious bodies all over you as we massage you with the pleasure that will blow your mind.\nWe know what you're looking for and we are here to give it to you.\nWe are the beauties you've been searching for.\nWe're not clock watchers like the other places.\nYou won't be disappointed when you see us,that's a promise.\nYou'll be even happier with the wonderful service we offer.\nstop by today and give us a try .\n\xa0\n\xa0\n\n\n\n\nWe are an agency and serve many cities. Photos are a likeness of the girls presented but may not be 100% accurate."

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
# model_name = "distilbert-base-uncased-finetuned-sst-2-english"
# model = AutoModelForSequenceClassification.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# With both the model and tokenizer initialized we are now able to get explanations on an example text.

from transformers_interpret import SequenceClassificationExplainer
cls_explainer = SequenceClassificationExplainer(
    model,
    tokenizer)
word_attributions = cls_explainer(text)

In [ ]:
cls_explainer.visualize("bert_vizNeg3.html")

In [ ]:
word_attributions

[('[CLS]', 0.0),
 ('77', -0.009758368134855139),
 ('##7', -0.07936665027486223),
 ('relax', 0.2990892238939921),
 ('spa', -0.12879330936212363),
 ('43', 0.12640561270436662),
 ('##10', -0.04830861860515412),
 ('w', 0.10651616208620975),
 ('ll', 0.27079037332212397),
 ('##lino', 0.39855714189588626),
 ('##is', 0.09783411423537268),
 ('ave', -0.03743073757948999),
 ('#', -0.09033844534034062),
 ('240', 0.026256813339543095),
 ('.', 0.016434362843513267),
 ('midland', -0.1690419568272774),
 ('.', 0.009402800759931669),
 ('tx', -0.3213072447533129),
 ('79', 0.06623658652743815),
 ('##70', -0.18208155651448163),
 ('##3', -0.17815650352317694),
 ('open', -0.030802192742160756),
 ('7', -0.048803319171009184),
 ('days', -0.02326182053718539),
 ('9', 0.005036160876784132),
 (':', 0.02008533454543746),
 ('00', 0.0812835574492703),
 ('##am', 0.06441275218950249),
 ('-', -0.06349643329400297),
 ('10', -0.07433490466786445),
 (':', 0.050315432791350374),
 ('00pm', 0.2952733802563707),
 ('tel', 0.18

In [ ]:
from transformers import TrainingArguments, Trainer, logging
import evaluate


# arguments for Trainer
test_args = TrainingArguments(
    output_dir = "./",
    do_train = False,
    do_predict = True,
    per_device_eval_batch_size = 1,
    gradient_accumulation_steps=256,
    eval_accumulation_steps=256,
    gradient_checkpointing=True,
    fp16=True,
    dataloader_drop_last = False    
)

# init trainer
trainer = Trainer(
              model = model, 
              eval_dataset=tokenized_datasets["test"],
              compute_metrics=compute_metrics,
              args = test_args)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using cuda_amp half precision backend


In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BartForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BartForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2998
  Batch size = 1


{'eval_loss': 0.5371596813201904,
 'eval_accuracy': 0.7248165443629087,
 'eval_f1': 0.0,
 'eval_runtime': 106.1354,
 'eval_samples_per_second': 28.247,
 'eval_steps_per_second': 28.247}